In [1]:
intention = '''Draft or create a class system to represent the pular entries. 
Ideally this will contain a way to nest entry objects under a root
'''
#%pip install docx
#%pip install python-docx #this mutates docx? 
#%pip install pydantic
#%pip install mypy
# %pip install numpy
from typing import Optional, Dict, List, Any, Union, Tuple
from pydantic import BaseModel, ValidationError, validator, root_validator, Field, constr
import json
import docx
from docx import Document
from datetime import datetime
import logging


In [2]:
#get current datetime
now = datetime.now()
current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")

#create file to save prints (use with jupyter magic enabled at the top of this cell: %%capture cap --no-stderr)
output_name = f"{current_time}_result.txt"
experiment = input("Enter emperiment description:")
print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

logger_filename = f"{current_time}.log"

# Creating an object
logger = logging.getLogger()

# Setting the threshold of logger to DEBUG
logger.setLevel(logging.ERROR)

#add encoding
handler = logging.FileHandler(logger_filename, 'w', 'utf-8') 
handler.setFormatter(logging.Formatter('%(asctime)s %(message)s'))
logger.addHandler(handler) 

# # Test messages
# logger.debug("Harmless debug Message")
# logger.info("Just an information")
# logger.warning("Its a Warning")
# logger.error("Did you try to divide by zero")
# logger.critical("Internet is down")

Experiment time: 2022-07-15_-_16-00-09
Experiment note: rare search but without X and numbers




In [3]:
# %%writefile f"docx_Paragraph_and_Runs_{current_time}.py"
def logger_root_validation_error_messages(e, logger_details, suppress, run_enumeration: Optional[int] = None) -> Union[RuntimeError, TypeError]:      
   #TODO add ability to handle assertion errors
   if run_enumeration is not None:
      run_num = f"|run#{run_enumeration}|" #type: ignore 
   else:
      run_num = ""
   try:
      for err in e.errors():
         if err['type'] in suppress['type'] or err['msg'] in suppress['msg']:
            logger.info(f"|SUPRESSED|{logger_details['function']}|{type(e)}|para#{logger_details['paragraph_enumeration']}{run_num}, with validation? error: {err}")
            return TypeError("suppressed Validation Error")
         else:
            logger.error(f"|unsuppressed|{logger_details['function']}|{type(e)}|para#{logger_details['paragraph_enumeration']}{run_num}, with validation? error: {err}")
            return TypeError("un-suppressed Validation Error")
   except:
      logger.error(f"|unsuppressed|{logger_details['function']}|{type(e)}|para#{logger_details['paragraph_enumeration']}{run_num}, with error: {e}")
      return RuntimeError("non-validation error")
   return RuntimeError("non-validation error")



class docx_Paragraph_and_Runs (BaseModel):
   """input:   paragraph = your_paragraph_here
   
   when given a docx document's paragraph object, will parse it to a specified schema
   """
   class Config:
      extra = 'allow'
      # arbitrary_types_allowed = True

   @root_validator(pre=True) #TODO Try have post validator for runs only?
   def _docx_structure_check(cls, values: Dict[str, Any]) -> Dict[str, Any]:
      new_values: Dict[str, Any] = {}
      para = values.get("paragraph",False)
      assert isinstance(para, eval('docx.text.paragraph.Paragraph')), 'please enter a docx paragraph assigned to the variable "paragraph", in the form of     paragraph = your_paragraph_here'
      
      paragraph_enumeration: int = values.get('paragraph_enumeration',None)
      assert isinstance(paragraph_enumeration, int), "assertion error, bad paragraph count/paragraph_enumeration value passed. Please pass an integer"
      new_values['paragraph_enumeration'] = paragraph_enumeration

      
      #setting up error and logger handling
      #suppress these errors
      suppress = {'type': ['value_error.any_str.min_length' #ignore zero length run_text, per run validator
                           ],
                  'msg': ['suppressed Validation Error'] #ignore suppressed errors earlier/lower in the stack      
      }
      #try to extract para features, 
      logger_details = {'function':'docx_Paragraph', 'paragraph_enumeration':paragraph_enumeration }
      try: 
         parsed_paras = docx_Paragraph(**{'paragraph':para}) #type: ignore
         for k,v in parsed_paras.dict().items():
            new_values[k] = v
      # except ValidationError as e:
      #    logger_root_validation_error_messages(e, logger_details, suppress)
      except BaseException as e:
         new_e = logger_root_validation_error_messages(e, logger_details, suppress)
         raise new_e

      #try to extract runs features
      logger_details = {'function':'docx_Run_List', 'paragraph_enumeration':paragraph_enumeration }    
      try:
         parsed_runs = docx_Run_List(**{'run_list':para.runs, 'paragraph_enumeration':paragraph_enumeration}) #type: ignore
         for k,v in parsed_runs.dict().items():
            new_values[k] = v
      except BaseException as e:
         new_e = logger_root_validation_error_messages(e, logger_details, suppress)
         raise new_e
         
      return new_values
      

   def interogate__para_text(self) -> str:
      # a = '__begin interogate__para_text method call__'
      # print(a)
      t = getattr(self, 'para_text', "")
      if not t:
         print("no para_text with:\n\t", self.dict())
      # else: print('final para text:\n\t',t)
      return t

   def paragraph_logger(self,level:int,msg:str,print_bool:bool):
      if print_bool:
         print(msg)
      else:
         logger.log(level,msg)


   def single_feature_identify(self,params:Dict['str',Any]) -> bool: 
      fl = isinstance(params['value'],float)
      bo = isinstance(params['value'],bool)
      if not fl and not bo:
         logger.error(f"error    |single_feature_identify||para#{enumerate}| provided parameters:  {params} have 'value' that is not [bool or float] as expected")
         return False
      enumeration : Optional[int] = getattr(self,"paragraph_enumeration",None)
      assert isinstance(enumeration, int),f"bad value for 'paragraph_enumeration' {enumeration}"
      list_from_runs: List[Optional[Union[float,bool]]] = getattr(self,params['docxFeature'],[None]) 

      mask: List[bool] = [True if x == params['value'] else False for x in list_from_runs]

      if any(mask) and len(mask)>1: #has bolds and is not len of 1
         if mask[0] is not True: ##has bolds not in the first position, and is not len of 1
            logger.warning(f"warning  |para#{enumeration}| paragraph has {params['strSummary']} text, but not in Run#1. Features are {repr(self)}")
         elif any(mask[1:]):
            logger.info(f"info      |para#{enumeration}| paragraph with {params['strSummary']} in multiple runs, including Run#1. #AUDIT these are resolved after aggregation. Features are {repr(self)}")
         else: pass
         return True
      elif any(mask):
         return True
      else:
         return False


In [4]:
class docx_Paragraph (BaseModel):
   """input:   paragraph = your_paragraph_here
   
   when given a docx document's paragraph object, will parse it to a specified schema
   """
   # docx_document_paragraph: Optional[Any] #This should be validated below. Left optional because its inclusion causes problems with default repr and serialization
   para_text: str = Field(..., min_length = 1) ##required, must be string, must be 1 long or more
   para_first_line_indent: Optional[int] = Field(...) #Required, but must be int OR none. https://pydantic-docs.helpmanual.io/usage/models/#required-optional-fields 
   para_left_indent: Optional[int] = Field(...) #Required, but must be int OR none. https://pydantic-docs.helpmanual.io/usage/models/#required-optional-fields 

   @root_validator(pre=True) #TODO Try have post validator for runs only?
   def _docx_structure_check(cls, values: Dict[str, Any]) -> Dict[str, Any]:
      para = values.get("paragraph",False)
      assert isinstance(para, eval('docx.text.paragraph.Paragraph')), 'please enter a docx paragraph assigned to the variable "paragraph", in the form of     paragraph = your_paragraph_here'

      new_values: Dict[str, Any] = {}
      #extract para features, 
      new_values['para_text'] = para.text #type: ignore
      new_values['para_first_line_indent'] = para.paragraph_format.first_line_indent #type: ignore
      new_values['para_left_indent'] = para.paragraph_format.left_indent #type: ignore

      return new_values


class docx_Run (BaseModel):
   """input:   run = your_run_here
   
   when given a docx document paragraphs run object, will parse it to a specified schema
   """
   run_text : str = Field(..., min_length = 1) #required, must be string, must be 1 long or more
   run_font_name : Optional[str] = Field(...) #required, must be string or None value
   run_font_size_pt : Optional[float] = Field(...)#Required, but must be float OR none value
   run_bold : Optional[bool] = Field(...) #Required, but must be bool OR none value
   run_italic : Optional[bool] = Field(...) #Required, but must be bool OR none value

   @root_validator(pre=True) #TODO Try have post validator for runs only?
   def _docx_structure_check(cls, values: Dict[str, Any]) -> Dict[str, Any]:
      run = values.get("run",False)
      assert isinstance(run, eval('docx.text.run.Run')), 'please enter a docx run assigned to the variable "run", in the form of     run = your_run_here'
      
      new_values : Dict[str, Any] = {}
      #loop through the runs in the paragraph and select the desired features
      new_values['run_text'] = run.text #type: ignore
      new_values['run_font_name'] = run.font.name #type: ignore
      if run.font.size is not None: #type: ignore
         new_values['run_font_size_pt'] = run.font.size.pt #type: ignore
      else: new_values['run_font_size_pt'] = None
      new_values['run_bold'] = run.bold #type: ignore
      new_values['run_italic'] = run.italic #type: ignore

      return new_values


class docx_Run_List (BaseModel):
   """input:   run_list = your_runs_in_a_list
   
   when given a list of docx document paragraphs run object, will parse it to a specified schema
   """
   #because the internals are validated, don't need to validate these other than that they were made into lists
   run_text : List[Any] = Field(...) #Required, must be list
   run_font_name : List[Any] = Field(...) #Required, must be list
   run_font_size_pt : List[Any] = Field(...) #Required, must be list
   run_bold : List[Any] = Field(...) #Required, must be list
   run_italic : List[Any] = Field(...) #Required, must be list

   @root_validator(pre=True) #TODO Try have post validator for runs only?
   def _docx_structure_check(cls, values: Dict[str, List[Any]]) -> Dict[str, Any]:
      from collections import defaultdict
      paragraph_enumeration = values.get('paragraph_enumeration',"<<FAILURE_paragraph_enumeration>>")
      runs = values.get("run_list",False)
      if not runs:
         raise ValueError('please enter a docx run list assigned to the variable "run_list", in the form of     run_list = your_run_list_here')
      new_values = defaultdict(list)
      suppress = {'type': ['value_error.any_str.min_length' #ignore zero length run_text, per run validator
                           ],
                  'msg': ['suppressed Validation Error'] #ignore suppressed errors earlier/lower in the stack      
      }
      logger_details = {'function':'parsed_run', 'paragraph_enumeration':paragraph_enumeration }
      
      for run_enumumeration, run in enumerate(runs): #type: ignore
         try:
            parsed_run = docx_Run(**{'run':run}) #this manner of root unpacking seems to give warnings since linter can't assess ahead of time
            assert isinstance(parsed_run, docx_Run), 'RUNTIME_ERR - the docx run object did not return the type expected'
            for k,v in parsed_run.dict().items():
               new_values[k].append(v) 

         except BaseException as e:
            new_e = logger_root_validation_error_messages(e, logger_details, suppress,run_enumeration=run_enumumeration)
            raise new_e
             
      return new_values

In [5]:
class Fula_Entry (BaseModel): 
   entity_word: str
   features: Optional[Dict[str,str]] = {}
   paragraphs_list: List[Any]
   paragraphs_extr = []
   sub_roots = []
   lemmas = []

In [6]:
%%capture cap --no-stderr

from collections import Counter
char_counts = Counter()


#Run docx module to parse the docx file
docx_filename = "Fula_Dictionary-repaired.docx"
# docx_filename = "pasted_docx page 1.docx"
document = Document(docx_filename)


# #get current datetime
# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")

# #create file to save prints (use with jupyter magic enabled at the top of this cell: %%capture cap --no-stderr)
# output_name = f"{current_time}_result.txt"
# experiment = input("Enter emperiment description:")
# print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

# logger_filename = f"{current_time}.log"

low_freq_odd_chars = [('\t', 72), ('`', 64), ('&', 49), ('!', 15), ('"', 14), ('=', 4), ('\n', 1), ('\xa0', 1),
('ù', 30), ('ï', 26),  ('ò', 8), ('Q', 4)]
low_freq_odd_chars = [x[0] for x in low_freq_odd_chars]
nums = list(range(10))
#('X', 25),
# low_freq_odd_chars.extend(nums)


for i, para in enumerate(document.paragraphs):

   try:
      entryObj = docx_Paragraph_and_Runs(**{'paragraph': para, 'paragraph_enumeration': i})
      # char_counts.update(entryObj.interogate__para_text())
      featureConfig = {
      'root': {'docxFeature': 'run_font_size_pt',
               'strSummary':'fontSize_12.0', 
               'value':12.0},
      'lemma': {'docxFeature': 'run_bold',
               'strSummary':'fontBold', 
               'value':True},
      }
      is_root = entryObj.single_feature_identify(featureConfig['root'])
      if is_root:
         print('\n\nroot at para number: ',i)
         paraText = entryObj.interogate__para_text()
         print('\t',paraText)

      is_lemma = entryObj.single_feature_identify(featureConfig['lemma'])
      if is_lemma:
         entryObj.interogate__para_text()
         paraText = entryObj.interogate__para_text()
         print('\t\tp#',i,'\t\t',paraText)
      
      p_text = entryObj.interogate__para_text()
      if not set(p_text).isdisjoint(low_freq_odd_chars):
         msg = 'rare_characters\t\t'+p_text
         entryObj.paragraph_logger(level=40,msg = msg, print_bool=False)
   except BaseException as e:
      if not e.args[0][0].exc.args[0] == 'suppressed Validation Error':
         print('\npara number: ',i)



In [10]:
with open(output_name, 'w', encoding="utf-8") as f:
    f.write(cap.stdout)

In [8]:
# char_counts
sorted_char_freq = sorted(char_counts.items(), key=lambda item: (-item[1], item[0]))
print(sorted_char_freq)
low_freq_odd_chars = [('\t', 72), ('`', 64), ('&', 49), ('!', 15), ('"', 14), ('=', 4), ('\n', 1), ('\xa0', 1),
('ù', 30), ('ï', 26),  ('ò', 8), ('Q', 4)]
#X is a source, and also shows up in some glosses
#('X', 25),
#numbers are used to cite certain sources with year ranges?
low_freq_odd_chars = [x[0] for x in low_freq_odd_chars]
nums = list(range(10))
# low_freq_odd_chars.extend(nums)
low_freq_odd_chars

[]


['\t', '`', '&', '!', '"', '=', '\n', '\xa0', 'ù', 'ï', 'ò', 'Q']

In [ ]:
#cleaning notes
# `new Kunari' - region in western Niger ; `nouveau Kounari' - région dans l'ouest du Niger
   #here the ` seems to be used at the beginning of a quotation, and a normal apostrophe at the end

In [9]:
# char_counts
sorted_char_val = sorted(char_counts.items(), key=lambda item: (item[0], -item[1]))
print(sorted_char_val)

[]
